In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import lightgbm as lgb
import catboost as cgb
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import r2_score

from skopt import BayesSearchCV
# parameter ranges are specified by one of below

from skopt.space import Real, Categorical, Integer

import xgboost as xgb
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold


In [2]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

#### Загрузка выборки

In [3]:
train = pd.read_parquet('data/train.parquet')
train.head()

,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,...,_YearBuiltBin_YearBin2,_YearBuiltBin_YearBin3,_YearBuiltBin_YearBin4,_YearBuiltBin_YearBin5,_YearBuiltBin_YearBin6,_YearBuiltBin_YearBin7,_YearRemodAddBin_YearBin4,_YearRemodAddBin_YearBin5,_YearRemodAddBin_YearBin6,_YearRemodAddBin_YearBin7
Id,,,,,,,,,,,,,,,,,,,,,
1,-0.127817,1.209296,0.625446,-0.355892,-0.339062,-0.472456,-0.805990,1.185669,0.548227,0.360672,...,0,0,0,0,0,1,0,0,0,1
2,0.120797,-0.805439,1.257846,-0.355892,0.003303,0.512947,0.433256,-0.867410,-0.378408,-0.054591,...,0,0,0,0,1,0,0,0,1,0
3,0.429834,1.137048,0.113946,-0.355892,0.231214,-0.317122,-0.575842,1.189908,0.680880,0.643806,...,0,0,0,0,0,1,0,0,0,1
4,0.110623,-0.805439,-0.513805,-0.355892,0.348751,-0.715166,-0.436663,1.148647,0.560689,0.804249,...,0,1,0,0,0,0,0,1,0,0
5,0.891805,1.429555,0.506871,-0.355892,0.296484,0.228976,0.122612,1.249303,1.314119,1.719716,...,0,0,0,0,0,1,0,0,0,1


In [4]:
test = pd.read_parquet('data/test.parquet')
test.head()

,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,...,_YearBuiltBin_YearBin2,_YearBuiltBin_YearBin3,_YearBuiltBin_YearBin4,_YearBuiltBin_YearBin5,_YearBuiltBin_YearBin6,_YearBuiltBin_YearBin7,_YearRemodAddBin_YearBin4,_YearRemodAddBin_YearBin5,_YearRemodAddBin_YearBin6,_YearRemodAddBin_YearBin7
Id,,,,,,,,,,,,,,,,,,,,,
1461,0.493221,-0.805439,0.072096,2.339152,-0.023846,-0.409352,-0.660217,-0.867410,-1.422865,1.219513,...,0,0,0,1,0,0,0,1,0,0
1462,0.892761,0.983593,1.129971,-0.355892,0.195355,0.675563,0.598430,-0.867410,-0.220639,-0.752989,...,0,0,0,1,0,0,0,1,0,0
1463,0.832144,-0.805439,0.823071,-0.355892,-0.387584,-0.297705,-0.548205,1.125708,0.400198,0.049224,...,0,0,0,0,0,1,0,0,0,1
1464,0.196046,0.355581,0.383646,-0.355892,0.074090,-0.302559,-0.555092,1.115578,0.353021,-0.007402,...,0,0,0,0,0,1,0,0,0,1
1465,-1.148225,-0.805439,-0.404529,-0.355892,0.689482,0.556635,0.478476,-0.867410,-0.335215,0.162478,...,0,0,0,0,0,1,0,0,0,1


In [5]:
test.shape

(1459, 337)

In [6]:
label = pd.read_parquet('data/label.parquet')
label.head()

,SalePrice
Id,
1,208500
2,181500
3,223500
4,140000
5,250000


In [7]:
label_log = label.apply(np.log)

In [8]:
import warnings

warnings.filterwarnings('ignore')

Предсказание будет сделано на основе ансамля моделей 

#### LightGBM Regressor

In [9]:
def bayesion_opt_lgbm(X, y, init_iter=5, n_iters=9, random_state=35, seed = 102, num_iterations = 100):
    dtrain = lgb.Dataset(data=X, label=y)
    def lgb_r2_score(preds, dtrain):
        labels = dtrain.get_label()
        return 'r2', r2_score(labels, preds), True
  # Objective Function
    def hyp_lgbm(num_leaves, feature_fraction, bagging_fraction, max_depth, min_split_gain, min_child_weight):

        params = {'application':'regression','num_iterations': 5000,
                  'learning_rate':0.05, 'early_stopping_round':60,
                  'metric':'l1'} # Default parameters
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, dtrain, nfold=5, seed=35, stratified=False, verbose_eval =None, metrics=['l2'])
        
        return -np.min(cv_result['l2-mean'])
    
    # Domain space-- Range of hyperparameters 
    pds = {'num_leaves': (80, 100),
            'feature_fraction': (0.1, 0.9),
            'bagging_fraction': (0.8, 1),
            'max_depth': (5, 25),
            'min_split_gain': (0.001, 0.1),
            'min_child_weight': (10, 25)
            }

    # Surrogate model
    optimizer = BayesianOptimization(hyp_lgbm, pds, random_state=random_state)

    # Optimize
    optimizer.maximize(init_points=init_iter, n_iter=n_iters)
    return optimizer

In [10]:
#Для более качественной оптимизации логарифмирую цену цену
opt = bayesion_opt_lgbm(train, label_log, init_iter=5, n_iters=10, random_state=77, seed = 101, num_iterations = 200)

|   iter    |  target   | baggin... | featur... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------
|  1        | -0.01476  |  0.9838   |  0.6138   |  20.07    |  12.09    |  0.009645 |  95.76    |
|  2        | -0.01494  |  0.8652   |  0.5329   |  9.805    |  18.18    |  0.04065  |  94.3     |
|  3        | -0.01534  |  0.9673   |  0.5708   |  10.92    |  14.22    |  0.07085  |  88.45    |
|  4        | -0.01548  |  0.8115   |  0.6976   |  14.05    |  12.64    |  0.005888 |  85.85    |
|  5        | -0.01567  |  0.8134   |  0.7009   |  6.275    |  16.48    |  0.03705  |  83.04    |
|  6        | -0.01396  |  0.8003   |  0.2887   |  22.28    |  23.16    |  0.0242   |  99.73    |
|  7        | -0.01474  |  1.0      |  0.1      |  25.0     |  25.0     |  0.1      |  93.65    |
|  8        | -0.01643  |  1.0      |  0.9      |  16.49    |  25.0     |  0.1      |  100.0    |
|  9        | -0.014

In [11]:
params = opt.max["params"]

params['max_depth'] = int(params['max_depth'])
params['num_leaves'] = int(params['num_leaves'])

In [12]:
regr = lgb.LGBMRegressor(**params)

In [13]:
regr.fit(train, label.apply(np.log))
pred_train = regr.predict(train)
pred_test = regr.predict(test)

In [14]:
scor_lgb = pd.DataFrame(index = test.index)
scor_lgb["SalePrice"] = [np.exp(x) for x in pred_test]

#### Catboost Regressor

In [15]:
def cat_hyp(depth, bagging_temperature): # Function to optimize depth and bagging temperature
    
    params = {"iterations": 100,
            "learning_rate": 0.05,
            "loss_function": "RMSE",
            "verbose": False} # Default Parameters
    params[ "depth"] = int(round(depth)) 
    params["bagging_temperature"] = bagging_temperature

    cat_feat = [] # Categorical features list
    cv_dataset = cgb.Pool(data=train,
                  label=label,
                  cat_features=cat_feat)

    scores = cgb.cv(cv_dataset,
              params,
              fold_count=3)
    
    return - np.min(scores['test-RMSE-mean']) 

In [16]:
pds = {'depth': (5, 8),
          'bagging_temperature': (3,10)
          }

In [17]:

# Surrogate model
optimizer = BayesianOptimization(cat_hyp, pds, random_state=2100)
                                  
# Optimize
optimizer.maximize(init_points=3, n_iter=7 )

|   iter    |  target   | baggin... |   depth   |
-------------------------------------------------
|  1        | -2.562e+0 |  8.931    |  5.069    |
|  2        | -2.621e+0 |  3.608    |  6.008    |
|  3        | -2.621e+0 |  6.422    |  5.9      |
|  4        | -2.734e+0 |  7.919    |  7.808    |
|  5        | -2.562e+0 |  4.26     |  5.276    |
|  6        | -2.621e+0 |  5.826    |  5.801    |
|  7        | -2.562e+0 |  10.0     |  5.0      |
|  8        | -2.621e+0 |  10.0     |  6.309    |
|  9        | -2.562e+0 |  7.75     |  5.013    |
|  10       | -2.562e+0 |  3.0      |  5.0      |


In [18]:
optimizer.max

{'target': -25615.237049828265,
 'params': {'bagging_temperature': 8.930629571990341,
  'depth': 5.06944936764366}}

In [19]:
parameters = optimizer.max['params']
parameters['depth'] = int(parameters['depth'])

In [20]:
regr = cgb.CatBoostRegressor(**parameters)

In [21]:
regr.fit(train, label)
pred_train = regr.predict(train)
pred_test = regr.predict(test)

Learning rate set to 0.04173
0:	learn: 74445.3743955	total: 3.55ms	remaining: 3.54s
1:	learn: 72326.6018582	total: 8.22ms	remaining: 4.1s
2:	learn: 70369.6911380	total: 12.1ms	remaining: 4.02s
3:	learn: 68477.2230349	total: 15.1ms	remaining: 3.77s
4:	learn: 66894.7231913	total: 17.9ms	remaining: 3.56s
5:	learn: 65067.6338193	total: 20.7ms	remaining: 3.42s
6:	learn: 63196.5959739	total: 24ms	remaining: 3.41s
7:	learn: 61530.1262667	total: 27.4ms	remaining: 3.4s
8:	learn: 60025.4964126	total: 30.1ms	remaining: 3.32s
9:	learn: 58508.0409345	total: 32.8ms	remaining: 3.25s
10:	learn: 57046.3089487	total: 35.7ms	remaining: 3.21s
11:	learn: 55547.9826649	total: 38.8ms	remaining: 3.19s
12:	learn: 54298.7722450	total: 41.8ms	remaining: 3.17s
13:	learn: 52861.1790154	total: 44.4ms	remaining: 3.13s
14:	learn: 51621.4756092	total: 47.6ms	remaining: 3.13s
15:	learn: 50479.5035431	total: 50.2ms	remaining: 3.09s
16:	learn: 49313.8314783	total: 52.9ms	remaining: 3.06s
17:	learn: 48225.6800173	total: 5

148:	learn: 18251.5232287	total: 551ms	remaining: 3.15s
149:	learn: 18187.1279327	total: 555ms	remaining: 3.14s
150:	learn: 18151.5137378	total: 559ms	remaining: 3.14s
151:	learn: 18105.6972669	total: 564ms	remaining: 3.15s
152:	learn: 18079.9928563	total: 568ms	remaining: 3.14s
153:	learn: 18036.3275935	total: 571ms	remaining: 3.13s
154:	learn: 18006.8416514	total: 574ms	remaining: 3.13s
155:	learn: 17981.6360681	total: 578ms	remaining: 3.12s
156:	learn: 17937.6155045	total: 583ms	remaining: 3.13s
157:	learn: 17910.9271699	total: 587ms	remaining: 3.13s
158:	learn: 17877.7618807	total: 590ms	remaining: 3.12s
159:	learn: 17868.8144211	total: 609ms	remaining: 3.19s
160:	learn: 17852.1026992	total: 612ms	remaining: 3.19s
161:	learn: 17828.7996450	total: 615ms	remaining: 3.18s
162:	learn: 17810.9431981	total: 617ms	remaining: 3.17s
163:	learn: 17780.9755550	total: 620ms	remaining: 3.16s
164:	learn: 17738.0158847	total: 624ms	remaining: 3.15s
165:	learn: 17702.4282787	total: 626ms	remaining

305:	learn: 14751.7156077	total: 1.15s	remaining: 2.6s
306:	learn: 14729.9967890	total: 1.15s	remaining: 2.59s
307:	learn: 14708.6071083	total: 1.15s	remaining: 2.59s
308:	learn: 14693.0649777	total: 1.16s	remaining: 2.58s
309:	learn: 14670.3482089	total: 1.16s	remaining: 2.59s
310:	learn: 14654.0695812	total: 1.17s	remaining: 2.59s
311:	learn: 14645.8011987	total: 1.17s	remaining: 2.58s
312:	learn: 14625.7715726	total: 1.18s	remaining: 2.58s
313:	learn: 14611.3363813	total: 1.18s	remaining: 2.58s
314:	learn: 14581.1457334	total: 1.18s	remaining: 2.57s
315:	learn: 14558.1174469	total: 1.19s	remaining: 2.57s
316:	learn: 14543.3266186	total: 1.19s	remaining: 2.56s
317:	learn: 14539.3372050	total: 1.19s	remaining: 2.56s
318:	learn: 14514.6088022	total: 1.2s	remaining: 2.55s
319:	learn: 14500.6430214	total: 1.2s	remaining: 2.55s
320:	learn: 14496.8831770	total: 1.2s	remaining: 2.54s
321:	learn: 14489.7127239	total: 1.2s	remaining: 2.54s
322:	learn: 14474.8757739	total: 1.22s	remaining: 2.5

471:	learn: 12365.3668960	total: 1.69s	remaining: 1.89s
472:	learn: 12354.2751867	total: 1.69s	remaining: 1.88s
473:	learn: 12344.2688632	total: 1.69s	remaining: 1.88s
474:	learn: 12327.8664978	total: 1.69s	remaining: 1.87s
475:	learn: 12309.1027766	total: 1.7s	remaining: 1.87s
476:	learn: 12293.9259315	total: 1.7s	remaining: 1.86s
477:	learn: 12288.8946822	total: 1.7s	remaining: 1.86s
478:	learn: 12282.8261294	total: 1.7s	remaining: 1.85s
479:	learn: 12275.5164676	total: 1.71s	remaining: 1.85s
480:	learn: 12259.9716906	total: 1.71s	remaining: 1.84s
481:	learn: 12249.9960893	total: 1.71s	remaining: 1.84s
482:	learn: 12231.4953943	total: 1.72s	remaining: 1.83s
483:	learn: 12223.3645871	total: 1.72s	remaining: 1.83s
484:	learn: 12207.5333709	total: 1.72s	remaining: 1.83s
485:	learn: 12201.6021180	total: 1.72s	remaining: 1.82s
486:	learn: 12189.2141194	total: 1.73s	remaining: 1.82s
487:	learn: 12174.2950002	total: 1.73s	remaining: 1.81s
488:	learn: 12168.7001771	total: 1.73s	remaining: 1.

637:	learn: 10579.9373448	total: 2.17s	remaining: 1.23s
638:	learn: 10578.6973517	total: 2.17s	remaining: 1.22s
639:	learn: 10563.4169379	total: 2.17s	remaining: 1.22s
640:	learn: 10550.1959647	total: 2.17s	remaining: 1.22s
641:	learn: 10537.6353218	total: 2.17s	remaining: 1.21s
642:	learn: 10527.7771908	total: 2.18s	remaining: 1.21s
643:	learn: 10513.5499972	total: 2.18s	remaining: 1.21s
644:	learn: 10501.7715816	total: 2.18s	remaining: 1.2s
645:	learn: 10497.6405610	total: 2.19s	remaining: 1.2s
646:	learn: 10483.2532955	total: 2.19s	remaining: 1.19s
647:	learn: 10471.6464137	total: 2.19s	remaining: 1.19s
648:	learn: 10467.5303325	total: 2.19s	remaining: 1.19s
649:	learn: 10457.7868985	total: 2.2s	remaining: 1.18s
650:	learn: 10451.9208000	total: 2.2s	remaining: 1.18s
651:	learn: 10443.2050517	total: 2.2s	remaining: 1.17s
652:	learn: 10437.6706927	total: 2.2s	remaining: 1.17s
653:	learn: 10425.6343233	total: 2.21s	remaining: 1.17s
654:	learn: 10421.5328874	total: 2.21s	remaining: 1.16

787:	learn: 9334.3946789	total: 2.58s	remaining: 696ms
788:	learn: 9321.7485733	total: 2.59s	remaining: 692ms
789:	learn: 9307.6081267	total: 2.59s	remaining: 689ms
790:	learn: 9298.2116869	total: 2.59s	remaining: 685ms
791:	learn: 9296.2247724	total: 2.6s	remaining: 682ms
792:	learn: 9280.9469509	total: 2.6s	remaining: 678ms
793:	learn: 9271.0366546	total: 2.6s	remaining: 675ms
794:	learn: 9262.5200741	total: 2.6s	remaining: 672ms
795:	learn: 9259.2452018	total: 2.61s	remaining: 668ms
796:	learn: 9258.3995736	total: 2.61s	remaining: 665ms
797:	learn: 9257.7629052	total: 2.61s	remaining: 662ms
798:	learn: 9252.0053667	total: 2.62s	remaining: 659ms
799:	learn: 9242.4919517	total: 2.62s	remaining: 655ms
800:	learn: 9237.2944903	total: 2.62s	remaining: 652ms
801:	learn: 9227.3869977	total: 2.63s	remaining: 649ms
802:	learn: 9214.7451298	total: 2.63s	remaining: 645ms
803:	learn: 9208.4199363	total: 2.63s	remaining: 642ms
804:	learn: 9201.3673177	total: 2.64s	remaining: 639ms
805:	learn: 91

938:	learn: 8254.7798024	total: 3.06s	remaining: 199ms
939:	learn: 8247.1959552	total: 3.07s	remaining: 196ms
940:	learn: 8242.6079427	total: 3.07s	remaining: 193ms
941:	learn: 8233.0474266	total: 3.07s	remaining: 189ms
942:	learn: 8220.2824878	total: 3.08s	remaining: 186ms
943:	learn: 8215.6728528	total: 3.08s	remaining: 183ms
944:	learn: 8209.4950318	total: 3.08s	remaining: 179ms
945:	learn: 8199.2780312	total: 3.08s	remaining: 176ms
946:	learn: 8192.6012730	total: 3.09s	remaining: 173ms
947:	learn: 8183.2275717	total: 3.09s	remaining: 170ms
948:	learn: 8175.4880372	total: 3.09s	remaining: 166ms
949:	learn: 8168.7880758	total: 3.1s	remaining: 163ms
950:	learn: 8161.3355636	total: 3.1s	remaining: 160ms
951:	learn: 8152.2131810	total: 3.1s	remaining: 157ms
952:	learn: 8147.5385654	total: 3.11s	remaining: 153ms
953:	learn: 8140.7835553	total: 3.11s	remaining: 150ms
954:	learn: 8132.6176943	total: 3.11s	remaining: 147ms
955:	learn: 8132.2464378	total: 3.12s	remaining: 143ms
956:	learn: 8

In [22]:
scor_cgb = pd.DataFrame(index = test.index)
scor_cgb["SalePrice"] = pred_test

CatBoosting Лучше себя показывает если цель не логарифмировать

#### XGBRegressor

Как ни странно именно коэффицент детерминации позволяет подогнать гипермпараметры наилучшим образом

In [23]:

# Define R2
def xgb_r2(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(preds, labels)
  
dtrain = xgb.DMatrix(train, label, feature_names=train.columns.values)

# Define Objective Function
def hyp_xgb(max_depth, subsample, colsample_bytree,min_child_weight, gamma ):
    params = {
    'n_estimators': 300,
    'eta': 0.05,
    'objective': 'reg:linear',
    'eval_metric':'mae', # Optional --> Use eval_metric if you want to stop evaluation based on eval_metric 
    'silent': 1
     }
    params['max_depth'] = int(round(max_depth))
    params['subsample'] = max(min(subsample, 1), 0)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['min_child_weight'] = int(min_child_weight)
    params['gamma'] = max(gamma, 0)
    scores = xgb.cv(params, dtrain, num_boost_round=1000,verbose_eval=False, early_stopping_rounds=10, feval=xgb_r2, maximize=True, nfold=5)
    return  -scores['test-mae-mean'].iloc[-1]

In [24]:
pds ={
  'min_child_weight':(14, 20),
  'gamma':(0, 5),
  'subsample':(0.5, 1),
  'colsample_bytree':(0.1, 1),
  'max_depth': (5, 10)
}

In [25]:

optimizer = BayesianOptimization(hyp_xgb, pds, random_state=10)
                                  
# Optimize

optimizer.maximize(init_points=5, n_iter=20)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------
[22:41:19] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:41:19] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:41:19] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:41:19] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language

|  4        | -1.412e+0 |  0.6513   |  3.609    |  6.459    |  19.51    |  0.8573   |
[22:41:33] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:41:33] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:41:33] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:41:33] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

|  8        | -1.389e+0 |  0.5169   |  0.6267   |  8.375    |  14.95    |  0.5867   |
[22:41:49] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:41:49] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:41:49] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:41:49] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

|  12       | -1.423e+0 |  0.9743   |  0.8065   |  6.323    |  19.21    |  0.8489   |
[22:42:04] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:42:04] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:42:04] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:42:04] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

|  16       | -1.408e+0 |  0.5717   |  4.339    |  7.736    |  18.62    |  0.7802   |
[22:42:19] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:42:19] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:42:19] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:42:19] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

|  20       | -1.407e+0 |  0.7942   |  2.233    |  7.499    |  16.94    |  0.8463   |
[22:42:32] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:42:32] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:42:32] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:42:32] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

|  24       | -1.422e+0 |  0.7633   |  4.751    |  8.943    |  19.75    |  0.6639   |
[22:42:46] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:42:46] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:42:46] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:42:46] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

In [26]:
optimizer.max

{'target': -13817.745312399999,
 'params': {'colsample_bytree': 0.5831164523882694,
  'gamma': 1.9830124103987823,
  'max_depth': 5.18956303609651,
  'min_child_weight': 15.301372552109147,
  'subsample': 0.6140849152343821}}

In [27]:
parameters =  optimizer.max['params']


parameters['max_depth'] = int(parameters['max_depth'])

In [28]:
regr = xgb.XGBRegressor(**parameters)

In [29]:
regr.fit(train, label)
pred_train = regr.predict(train)
pred_test = regr.predict(test)

In [30]:
scor_xgb = pd.DataFrame(index = test.index)
scor_xgb["SalePrice"] = pred_test

#### Random Forest Regressor

In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [46]:
parameters = {
   'n_estimators': [100, 200, 400],
   'max_features': ['auto', 'sqrt', 'log2'],
   'max_depth' : [4,8,16,24,32],
   'criterion' : ['mse', 'mae'],
    'min_impurity_decrease' : [0.0, 1e-6, 1e-5, 1e-4, 1e-3]
}


In [47]:
grid = RandomizedSearchCV(
    RandomForestRegressor(),
    parameters,
    n_iter = 150,
    n_jobs = -1,
    cv = 3,
    scoring =  "neg_mean_squared_log_error",
    verbose = 1
)

In [48]:
grid.fit(train, label)

Fitting 3 folds for each of 150 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:   46.2s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 438 tasks      | elapsed: 19.9min
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed: 21.5min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=150,
                   n_jobs=-1,
                   param_distributions={'criterion': ['mse', 'mae'],
                                        'max_depth': [4, 8, 16, 24, 32],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_impurity_decrease': [0.0, 1e-06,
                                                                  1e-05, 0.0001,
                                                                  0.001],
                                        'n_estimators': [100, 200, 400]},
                   scoring='neg_mean_squared_log_error', verbose=1)

In [49]:
grid.best_score_

-0.018468768992907055

In [50]:
regr = RandomForestRegressor(max_depth = 24, min_impurity_decrease = 1e-5)

In [51]:
regr.fit(train, label)
pred_train = regr.predict(train)
pred_test = regr.predict(test)

In [52]:
scor_rf = pd.DataFrame(index = test.index)
scor_rf["SalePrice"] = pred_test

In [53]:
scor_rf.to_csv('rf.csv')

#### Submission

Чтобы уменьшить влияние возможных выбросов используем логарифм

In [54]:
seria = (scor_lgb["SalePrice"].apply(np.log) * 2 +\
         scor_cgb["SalePrice"].apply(np.log) +\
         scor_xgb["SalePrice"].apply(np.log) +\
         scor_rf["SalePrice"].apply(np.log)) / 5

In [55]:
seria = seria.apply(np.exp)

In [56]:
scor_all = pd.DataFrame(index = test.index)
scor_all["SalePrice"] = seria

In [57]:
scor_all.to_csv("submission.csv")